In [1]:
#Includes
import zipfile
import pickle
import pandas as pd
import tkinter as tk
import numpy as np
import os
import random

In [2]:
def get_data(n, events_file, data_file):
    # Read the 'events' CSV file
    events_df = pd.read_csv(events_file)

    # Filter rows where column 2 is 1
    filtered_df = events_df[events_df.iloc[:, n] == 1]

    # Randomly select a row and get the value in column 1 (assuming column names are integers)
    random_row = filtered_df.sample()
    id = random_row.iloc[0, 0]

    # Read the 'data' CSV file
    data_df = pd.read_csv(data_file)

    # Find the row with the id in column 1 and get the corresponding data in the specified columns
    x = data_df[data_df.iloc[:, 0] == id][["C3", "Cz", "C4", "P3", "Pz", "P4"]].values.tolist()

    return id, x


In [3]:
# Get all files in the 'train/train' directory
def get_file(file):
    files = os.listdir(file)

    # Randomly select a base file (one ending with '_events.csv')
    events_file = random.choice([file for file in files if '_events.csv' in file])

    # Construct the corresponding file name
    data_file = events_file.replace('_events.csv', '_data.csv')
    return events_file, data_file


In [4]:
from torch import nn


class NNet(nn.Module):
    def __init__(self, in_channels=6, out_channels=6):
        super(NNet, self).__init__()
        self.hidden = 32
        self.net = nn.Sequential(
            nn.Conv1d(in_channels, in_channels, 5, padding=2),
            nn.Conv1d(in_channels, self.hidden, 16, stride=16),
            nn.SiLU(),
            nn.Conv1d(self.hidden, self.hidden, 7, padding=3),
        )
        for i in range(6):
            self.net.add_module('conv{}'.format(i), \
                                self.__block(self.hidden, self.hidden))
        self.net.add_module('final', nn.Sequential(
            nn.Conv1d(self.hidden, 12, 1),
            nn.Conv1d(12, out_channels, 1),
            nn.Sigmoid()
        ))
        
    def __block(self, inchannels, outchannels):
        return nn.Sequential(
            nn.MaxPool1d(2, 2),
            nn.Conv1d(inchannels, outchannels, 5, padding=2),
            nn.SiLU(),
            nn.Conv1d(outchannels, outchannels, 5, padding=2),
            nn.SiLU()
          )
    
    def forward(self, x):
        return self.net(x)

In [5]:
import pickle

# Load the model from file
with open('model.p', 'rb') as file:
    model = pickle.load(file)


In [8]:
class EEGDataViewer:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("EEG Data Viewer")
        self.n = None
        self.buttons = []
        self.back_button=None
        self.id_label = None
        self.label = None
        self.x_label = None

    def create_buttons(self):
        button_texts = ["Button 1", "Button 2", "Button 3", "Button 4", "Button 5", "Button 6"]
        for i in range(6):
            button = tk.Button(self.window, text=button_texts[i], command=lambda i=i: self.set_n(i+1))
            self.buttons.append(button)
            button.pack()

    def set_n(self, button_number):
        for button in self.buttons:
            button.destroy()

        self.n = button_number
        self.label = tk.Label(self.window, text=f"n = {self.n}")
        self.id_label = tk.Label(self.window, text="id = ")
        self.x_label = tk.Label(self.window, text="x = ")
        self.label.pack()
        self.id_label.pack()
        self.x_label.pack()

        events_file, data_file = self.get_file('train/train')
        id, x = self.get_data(self.n, 'train/train/' + events_file, 'train/train/' + data_file)
        self.id_label.config(text=f"id = {id}")
        self.x_label.config(text=f"x = {x}")
        self.back_button = tk.Button(self.window, text="Back", command=self.back_to_buttons)
        self.back_button.pack()

    def back_to_buttons(self):
        self.label.destroy()
        self.id_label.destroy()
        self.x_label.destroy()
        self.create_buttons()
        self.back_button.destroy()

    def get_file(self, file):
        files = os.listdir(file)
        events_file = random.choice([file for file in files if '_events.csv' in file])
        data_file = events_file.replace('_events.csv', '_data.csv')
        return events_file, data_file

    def get_data(self, n, events_file, data_file):
        events_df = pd.read_csv(events_file)
        filtered_df = events_df[events_df.iloc[:, n] == 1]
        random_row = filtered_df.sample()
        id = random_row.iloc[0, 0]
        data_df = pd.read_csv(data_file)
        data = data_df[data_df.iloc[:, 0] == id][["C3", "Cz", "C4", "P3", "Pz", "P4"]].values.tolist()
        data= data.T.astype(np.float32)
        print(Model_Predict(data))
        
        return id, data
    
    
    def Model_Predict(d):
        Predicted = model.predict(d)
        return Predicted

    def start(self):
        self.create_buttons()
        self.window.mainloop()

# Create an instance of the EEGDataViewer class and start the application
eeg_viewer = EEGDataViewer()
eeg_viewer.start()
